In [1]:
import requests
import pandas as pd
import numpy as np
import time
import json
from bs4 import BeautifulSoup

In [10]:
# Load the initial dataframe
df = pd.read_csv("file_path")
#df = df.head(10)  # Use the first 10 rows for testing

In [3]:
# Create Zillow URL column
def create_zillow_url(row):
    street_part = '-'.join(row['street'].split())
    city_part = '-'.join(row['City'].split())
    url = f"https://www.zillow.com/homes/{street_part}-{city_part},-CA-{row['ZIP Code']}/"
    return url

# Add Zillow URL column to df
df['zillow_url'] = df.apply(create_zillow_url, axis=1)

In [11]:
# Initialize empty columns if they don't exist
for col in ['year_built', 'lot_size']:
    if col not in df.columns:
        df[col] = np.nan # Fill with null

In [13]:
# Function to scrape property data from Zillow with retries
def get_property_details(url, retries=2, delay=5):
    payload = {
        'source': 'universal',
        'url': url,
        'user_agent_type': 'desktop',
    }
    # Loop for scraping attempts
    for attempt in range(retries):
        # Try / Except for errors so code continues
        try:
            response = requests.post(
                'https://realtime.oxylabs.io/v1/queries',
                auth=('USERNAME', 'PASSWORD'),
                json=payload,
                timeout=30
            )
            response.raise_for_status()
            content = response.json().get('results', [{}])[0].get('content', '')

            if content:
                soup = BeautifulSoup(content, 'html.parser')
                script_tag = soup.find('script', {'id': '__NEXT_DATA__'})
                
                if script_tag:
                    json_data = json.loads(script_tag.string)
                    gdp_cache_str = json_data['props']['pageProps']['componentProps']['gdpClientCache']
                    gdp_cache = json.loads(gdp_cache_str)
                    
                    property_info = None
                    for key, value in gdp_cache.items():
                        if 'property' in value:
                            property_info = value['property']
                            break
                    
                    year_built = property_info.get('yearBuilt', 'N/A') if property_info else 'N/A'
                    lot_size = property_info.get('lotSize', 'N/A') if property_info else 'N/A'
                    return year_built, lot_size
            return 'N/A', 'N/A'  # Return default values if no content
        
        except (requests.exceptions.RequestException, Exception) as e:
            print(f"Error fetching data for {url}: {e}")
            if attempt < retries - 1:
                time.sleep(delay)  # Wait before retrying
            else:
                return 'N/A', 'N/A'  # Return default values after max retries

In [14]:
# Iterate over the dataframe row by row
for i in range(len(df)):
    try:
        # Check if the row needs updating
        if pd.isna(df.iloc[i]['year_built']) or df.iloc[i]['year_built'] == 'N/A' or \
           pd.isna(df.iloc[i]['lot_size']) or df.iloc[i]['lot_size'] == 'N/A':
            # Fetch property details
            year_built, lot_size = get_property_details(df.iloc[i]['zillow_url'])
            
            # Update the row in the DataFrame
            df.at[i, 'year_built'] = year_built
            df.at[i, 'lot_size'] = lot_size
            
            # Print an update for every 300th record
            if (i + 1) % 300 == 0:  # i + 1 because i is zero-indexed
                # Create a dynamic filename with the range of indexes processed
                start_index = 1  # Start index for the first batch
                end_index = i + 1  # Current index + 1 (inclusive)
                filename = f"/Users/hadenloveridge/Desktop/AML/Final_Project/split_data/sub_file_4/updated_houses_{start_index}_to_{end_index}.csv"
    
                # Save intermediate progress
                df.to_csv(filename, index=False)
                print(f"Intermediate save at {i + 1} records: {filename}")
                
    except Exception as e:
        print(f"Error processing record {i + 1}: {e}")

# Save the final DataFrame to a CSV
df.to_csv('saved_file_path', index=False)
print("All records processed and saved.")

Error fetching data for https://www.zillow.com/homes/2173-Eastridge-Trail-Oxnard,-CA-93036.0/: 'componentProps'
Error fetching data for https://www.zillow.com/homes/885-Laramie-Court-Newbury-Park,-CA-91320.0/: HTTPSConnectionPool(host='realtime.oxylabs.io', port=443): Read timed out. (read timeout=30)
Intermediate save at 3600 records: /Users/hadenloveridge/Desktop/AML/Final_Project/split_data/sub_file_4/updated_houses_1_to_3600.csv
Error fetching data for https://www.zillow.com/homes/Address-not-provided-Camarillo,-CA-nan/: 'componentProps'
Error fetching data for https://www.zillow.com/homes/Address-not-provided-Camarillo,-CA-nan/: 'componentProps'
Error fetching data for https://www.zillow.com/homes/Address-not-provided-Camarillo,-CA-nan/: 'componentProps'


/var/folders/rx/yr5d88794lg__8cc26qmp1sw0000gn/T/ipykernel_93777/3018124338.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'year_built'] = year_built
/var/folders/rx/yr5d88794lg__8cc26qmp1sw0000gn/T/ipykernel_93777/3018124338.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'lot_size'] = lot_size


Error fetching data for https://www.zillow.com/homes/2338-Nicklaus-St.-Oxnard,-CA-93036.0/: 'gdpClientCache'
Error fetching data for https://www.zillow.com/homes/2338-Nicklaus-St.-Oxnard,-CA-93036.0/: 'gdpClientCache'
Error fetching data for https://www.zillow.com/homes/2338-Nicklaus-St.-Oxnard,-CA-93036.0/: 'gdpClientCache'
Error fetching data for https://www.zillow.com/homes/148-Seaspray-Way-Port-Hueneme,-CA-93041.0/: 'gdpClientCache'
Error fetching data for https://www.zillow.com/homes/148-Seaspray-Way-Port-Hueneme,-CA-93041.0/: 'gdpClientCache'
Error fetching data for https://www.zillow.com/homes/148-Seaspray-Way-Port-Hueneme,-CA-93041.0/: 'gdpClientCache'
Error fetching data for https://www.zillow.com/homes/665-Camino-De-La-Luz-Newbury-Park,-CA-91320.0/: 'gdpClientCache'
Error fetching data for https://www.zillow.com/homes/665-Camino-De-La-Luz-Newbury-Park,-CA-91320.0/: 'gdpClientCache'
Error fetching data for https://www.zillow.com/homes/665-Camino-De-La-Luz-Newbury-Park,-CA-9132